In [141]:
import pandas as pd
import numpy as np
import datetime
from faker import Faker
import warnings
warnings.filterwarnings('ignore')

In [171]:
class ChargeGeneration:
    def __init__(self,costed_df,product_df,pi_df):
        self.costed_df = costed_df
        self.product_df = product_df
        self.pi_df = pi_df
        
        self.charge = pd.DataFrame(columns = ["charge_id",
                                              "business_product_instance_id",
                                              "charge_counter",
                                              "date",
                                              "cost",
                                              "event_type"]) # Финальный этап
    
    def generate_for_one_instance(self, product_instance_id):
        event_for_instance_id = costed_df[costed_df.business_product_instance_id == product_instance_id].sort_values(by="date") 
        less_event_data = event_for_instance_id[["date","cost"]].copy()
        cost_for_product_by_instance_id = product_df[product_df.product_id == int(pi_df[pi_df.business_product_instance_id == product_instance_id].product_id)].total_cost
        
        business_product_instance_id = []
        charge_counter = [] # Counter for recurrent charge. You can use it as technical field for generation
        date = [] # Date of the event
        cost = [] # Cost for charge
        event_type = [] # Charge type. One-time or recurrent
        
        instance_charges = self._charge_by_events(less_event_data)
        #print(instance_charges.date)
        for index,rows in instance_charges.iterrows():
            business_product_instance_id.append(product_instance_id)
            charge_counter.append(float(rows.cost))
            cost.append(float(rows.cost) + float(cost_for_product_by_instance_id.values[0]))
            date.append(str(rows.date.date()))
            event_type.append("recurrent")
            
        
        return business_product_instance_id, charge_counter, date, cost, event_type
        
    
    def _charge_by_events(self,less_event_data):
        less_event_data.date = pd.to_datetime(less_event_data['date'],format="%Y/%m/%d")
        #less_event_data.date = less_event_data['date'].dt.date
        less_event_data = less_event_data.set_index('date')    
        sum_by_m = less_event_data.groupby(pd.Grouper(freq='M'))['cost'].sum().reset_index()
        return sum_by_m
    
    def charge_for_instance(self,product_instance_id):
        charge = self.generate_for_one_instance(product_instance_id)
        for i in range(len(charge[0])):
            
            instance = ["None", charge[0][i], charge[1][i], charge[2][i], charge[3][i], charge[4][i]]
            df_to_add = pd.DataFrame([instance],columns=self.charge.columns.values)
            
            self.charge = self.charge.append(df_to_add,ignore_index=True)
        return charge
    
    def generate_all(self):
        for i in range(costed_df.business_product_instance_id.nunique()):
            self.charge_for_instance(i)
        self.charge.charge_id = np.arange(0,self.charge.shape[0])
    
    def save_to_csv(self, file_name="charge.csv"):
        """Saves generated data to csv table.
           :param file_name: The name of the file to be written to, defaults to charge.csv
           :type file_name: str
        """
        
        self.charge.to_csv(file_name,index=False)

In [172]:
cg = ChargeGeneration(costed_df,product_df,pi_df)

In [173]:
%%time
cg.generate_all()

Wall time: 3min 15s


In [174]:
cg.charge.head(10)

,charge_id,business_product_instance_id,charge_counter,date,cost,event_type
0,0,0,43395.0,2020-03-31,50600.0,recurrent
1,1,0,39273.3,2020-04-30,46478.3,recurrent
2,2,0,50006.0,2020-05-31,57211.0,recurrent
3,3,0,57253.9,2020-06-30,64458.9,recurrent
4,4,0,53082.7,2020-07-31,60287.7,recurrent
5,5,0,49781.6,2020-08-31,56986.6,recurrent
6,6,0,53540.3,2020-09-30,60745.3,recurrent
7,7,0,50129.2,2020-10-31,57334.2,recurrent
8,8,0,43535.8,2020-11-30,50740.8,recurrent
9,9,0,46184.6,2020-12-31,53389.6,recurrent


In [175]:
cg.save_to_csv()

In [156]:
test.date = pd.to_datetime(test['date'],format="%Y/%m/%d")

KeyError: 'date'

In [35]:
test.date = test['date'].dt.date

In [150]:
cost_for_product_by_instance_id = product_df[product_df.product_id == int(pi_df[pi_df.business_product_instance_id == 0].product_id)].total_cost
cost_for_product_by_instance_id.values[0]       

7205

In [51]:
test = test.set_index('date')

In [155]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10220 entries, 2020-03-07 to 2022-03-06
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   cost    10220 non-null  float64
dtypes: float64(1)
memory usage: 159.7 KB


In [161]:
event_for_instance_id = costed_df[costed_df.business_product_instance_id == 0].sort_values(by="date") 
less_event_data = event_for_instance_id[["date","cost"]].copy()
less_event_data.date = pd.to_datetime(less_event_data['date'],format="%Y/%m/%d")
#less_event_data.date = less_event_data['date'].dt.date
less_event_data = less_event_data.set_index('date')    
sum_by_m = less_event_data.groupby(pd.Grouper(freq='M'))['cost'].sum().reset_index()
str(sum_by_m.date[0].date())

'2020-03-31'

In [52]:
sum_by_m = test.groupby(pd.Grouper(freq='M'))['cost'].sum().reset_index()

In [57]:
sum_by_m

,date,cost
0,2020-03-31,43395.0
1,2020-04-30,39273.3
2,2020-05-31,50006.0
3,2020-06-30,57253.9
4,2020-07-31,53082.7
5,2020-08-31,49781.6
6,2020-09-30,53540.3
7,2020-10-31,50129.2
8,2020-11-30,43535.8
9,2020-12-31,46184.6


In [6]:
customer_df = pd.read_csv("Customer.csv",index_col = False)
customer_df

,ID,first_name,last_name,date_of_birth,gender,email,MSISDN,agree_for_promo,autopay_card,customer_category,language,customer_since,region,status
0,0,さゆり,山崎,2004-05-26,M,C*******8@gmail.com,90-7084-4635,No,Yes,physical,Japanese,2020-03-07,Kantō,active
1,1,直子,渡辺,1993-10-17,F,O*******5@yahoo.com,90-5460-3945,No,Yes,physical,Japanese,2010-11-21,Kantō,active
2,2,舞,青木,1994-01-03,M,v*******5@hotmail.com,90-8598-0092,No,Yes,physical,Japanese,2011-02-05,Kansai(Kinki),inactive(debt)
3,3,舞,鈴木,1987-12-24,F,N*******9@yahoo.com,90-8353-5522,No,Yes,physical,Japanese,2016-06-17,Kyūshū & Okinawa,active
4,4,学,中村,1951-03-16,F,R*******4@hotmail.com,90-7877-8066,No,No,physical,Japanese,2021-10-04,Kantō,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,裕太,鈴木,1999-06-06,M,f*******0@hotmail.com,90-8957-3148,No,Yes,physical,Japanese,2018-01-24,Kantō,active
9996,9996,翔太,山田,1997-12-24,M,V*******6@hotmail.com,90-0570-4063,No,Yes,physical,Japanese,2022-02-27,Tōhoku,active
9997,9997,康弘,鈴木,1968-10-21,F,i*******8@gmail.com,90-5248-6057,No,Yes,physical,Japanese,2011-07-14,Chūbu,active
9998,9998,晃,村上,1957-01-09,F,D*******2@yahoo.com,90-9060-1773,No,Yes,physical,Japanese,2019-05-20,Kansai(Kinki),active


In [2]:
pd.__version__

'1.4.1'

In [7]:
pi_df = pd.read_csv("product_instance.csv",index_col = False)
pi_df

,business_product_instance_id,customer_id,product_id,activation_date,termination_date,Status,distribution_channel
0,0,0,4,2020-03-07,NaN,Active,online
1,1,1,5,2018-02-16,2021-05-25,Inactive,online
2,2,1,6,2021-05-25,NaN,Active,physical_shop
3,3,2,5,2011-12-13,2020-02-24,Inactive,physical_shop
4,4,2,6,2020-02-24,NaN,Active,online
...,...,...,...,...,...,...,...
23883,23883,9998,7,2018-12-19,2022-01-06,Inactive,physical_shop
23884,23884,9998,5,2022-01-06,NaN,Active,online
23885,23885,9998,10,2022-01-06,NaN,Active,online
23886,23886,9999,12,2014-03-24,2020-06-27,Inactive,physical_shop


In [8]:
product_df = pd.read_csv("product.csv",index_col = False)
product_df

,product_id,product_name,product_category,product_type,recurrent,cost_for_call,cost_for_sms,cost_for_data,allowance_sms,allowance_voice,allowance_data,total_cost
0,1,5G Gigaho Premier,5G,tariff,regularly,22,3.3,0,NaN,NaN,unlimited,7315
1,2,Kake-hodai Option,5G,addon,regularly,0,0.0,0,NaN,unlimited,NaN,1870
2,3,5G Gigalight,5G,tariff,regularly,22,3.3,0,NaN,NaN,7,6765
3,4,Gigaho Premier,LTE,tariff,regularly,22,3.3,0,NaN,NaN,60,7205
4,5,Gigalight,LTE,tariff,regularly,22,3.3,0,NaN,NaN,7,6765
5,6,Hajimete Sumaho Plan,5G-LTE,tariff,regularly,0,3.3,0,NaN,NaN,1,1980
6,7,Kake-hodai Option (¥1000),5G-LTE,addon,regularly,0,0.0,0,NaN,unlimited,NaN,1100
7,8,U15 Hajimete Sumaho Plan,5G-LTE,tariff,regularly,22,3.3,0,NaN,NaN,5,1980
8,9,5G Data Plus,data only (5G),addon,regularly,0,0.0,0,NaN,NaN,30,1100
9,10,Data Plus,data only (LTE),addon,regularly,0,0.0,0,NaN,NaN,30,1100


In [9]:
costed_df = pd.read_csv("costed_event.csv",index_col = False)
costed_df

,Unnamed: 0,event_id,business_product_instance_id,date,hour,minuts,cost,duration,number_of_sms,total_volume,event_type,direction,roaming,calling_msisdn,called_msisdn
0,0,0,0,2021-04-22,16,43,418.0,1160,0,19,Call,In,No,90-3236-4494,90-7084-4635
1,1,1,0,2020-11-24,9,33,0.0,297,0,1080,Data,NaN,No,NaN,NaN
2,2,2,0,2020-10-30,14,17,0.0,2501,0,1453,Data,NaN,Yes,NaN,NaN
3,3,3,0,2021-02-20,3,17,0.0,1823,0,1450,Data,NaN,No,NaN,NaN
4,4,4,0,2020-07-14,21,10,594.0,1624,0,27,Call,Out,No,90-7084-4635,90-9271-3398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1660857,1660857,1660857,99,2018-08-20,7,36,0.0,4541,0,1325,Data,NaN,No,NaN,NaN
1660858,1660858,1660858,99,2015-11-01,23,48,0.0,1353,0,22,Call,Out,No,90-6395-5974,90-7885-8795
1660859,1660859,1660859,99,2017-09-18,16,55,0.0,1,1,39,SMS,In,No,90-9373-9423,90-6395-5974
1660860,1660860,1660860,99,2017-05-10,17,26,0.0,2421,0,40,Call,Out,No,90-6395-5974,90-9947-6796


In [60]:
costed_df.business_product_instance_id.nunique()

100